Defining data frame variables 

In [1]:
import pandas as pd
rowList = open("../data/train_self_original.txt", "r").readlines()
conversations = {
    "conv_id": []
}
chats = {
    "conv_id": [],
    "buddy1": [],
    "chat_id": [],
    "buddy2": []
}
personals = {
    "persona": [],
    "personaId": [],
    "conv_id": []
}
answers = {
    "conv_id": [],
    "chat_id": [],
    "ans_id": [],
    "answer": []
}
isPersona = lambda rowContext : rowContext.find("your persona:")!=-1


In [2]:
def parseAnswer(conv_id, chat_id, answersContext):
    answersList = answersContext.split("|")
    for i in range(0, len(answersList)):
        answers["conv_id"].append(conv_id)
        answers["chat_id"].append(chat_id)
        answers["ans_id"].append(len(answers["ans_id"])+1)
        answers["answer"].append(answersList[i])        


In [3]:
def parseChat(currentContext, conv_id):
    buddy1EndPointPattern = "\t"
    buddy1EndPoint = currentContext.index(buddy1EndPointPattern)
    buddy2EndPointPattern ="\t\t"
    buddy2EndPoint = currentContext.index(buddy2EndPointPattern)
    buddy1str = currentContext[2:buddy1EndPoint]
    chats["conv_id"].append(conv_id)
    chats["buddy1"].append(buddy1str)
    chats["chat_id"].append(len(chats["chat_id"])+1)
    chats["buddy2"].append(currentContext[buddy1EndPoint+len(buddy1EndPointPattern):buddy2EndPoint])
    answersContext = currentContext[buddy2EndPoint+len(buddy2EndPointPattern):]
    parseAnswer(conv_id, chats["chat_id"][-1], answersContext)
    


In [4]:
def parsePersona(conv_id, persona):
    personals["persona"].append(slicePersona(persona))
    personals["personaId"].append(len(personals["personaId"])+1)
    personals["conv_id"].append(conv_id)

def slicePersona(persona) :
    pattern = "your persona:"
    prefixInd = persona.rindex(pattern)
    retInd = persona.index("\n")
    return persona[prefixInd+len(pattern):retInd]

In [5]:
def parseConversation(i):
    personaLen=0
    personaIt = i
    while(isPersona(rowList[personaIt]) and personaIt < len(rowList)):
        personaLen+=1
        personaIt+=1
    conversations["conv_id"].append(len(conversations["conv_id"])+1)
    
    for persona in rowList[i:i+personaLen] :
        parsePersona(conversations["conv_id"][-1], persona)
    return personaLen-1
    

Start converting information from txt file and creating dataframes

In [6]:
def parseObjects():
    skipPersona = lambda index, personaLen : index+int(personaLen)
    i = 0
    
    while i < len(rowList):
        if isPersona(rowList[i]) :
            personaLen = parseConversation(i)
            i = skipPersona(i, personaLen)
        else:
            parseChat(rowList[i], conversations["conv_id"][-1])
        i+=1

parseObjects()
convdf = pd.DataFrame(conversations, columns=["conv_id"])
personadf = pd.DataFrame(personals, columns=["conv_id", "persona","personaId"])
chatdf = pd.DataFrame(chats, columns=["conv_id", "buddy1", "chat_id", "buddy2"])
answerdf = pd.DataFrame(answers, columns = ["conv_id", "chat_id", "ans_id", "answer"])

Getting the most common personal informations

In [14]:
# personadf
personadf["persona"].value_counts().idxmax


<bound method Series.idxmax of  my favorite color is blue.                                                    225
 my favorite food is pizza.                                                    168
 i am a vegan.                                                                 116
 i love animals.                                                               110
 i like to read.                                                               109
 my favorite color is red.                                                     108
 i'm a vegan.                                                                  104
 i love dogs.                                                                   93
 my favorite color is purple.                                                   92
 i love to travel.                                                              86
 i love to read.                                                                85
 i like to cook.                                        

In [13]:
# chatdf
# chatdf["buddy1"].value_counts().idxmax
chatdf["buddy2"].value_counts().idxmax


<bound method Series.idxmax of what do you do for a living ?                                                     195
hi how are you today                                                              147
hi how are you today ?                                                             83
do you have any hobbies ?                                                          78
how are you doing today                                                            76
hi how are you doing                                                               73
hello how are you today ?                                                          72
hello , how are you today ?                                                        72
hi how are you doing ?                                                             70
what city are you from ?                                                           59
what do you do for work ?                                                          57
hi , how are you today 

In [9]:
# answerdf